<a href="https://colab.research.google.com/github/akshayonly/Mini-Projects/blob/main/Spotify_Radar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Know Your Music Taste (KYMT)
- Spotify Edition
- BETA.1.07.21

In [ ]:
!pip3 install spotipy
!pip3 install ColorThief

In [ ]:
import spotipy as sp
from colorthief import ColorThief
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import PIL 
import urllib 
import sys
import io

In [ ]:
def get_dataFrame(results, sp_access):
  # create a list of song ids
  ids = []

  for item in results['tracks']['items']:
          track = item['track']['id']
          ids.append(track)
          
  song_meta = {'id':[],'album':[], 'name':[], 
               'artist':[],'explicit':[],'popularity':[]}

  for song_id in ids:
      # get song's meta data
      meta = sp_access.track(song_id)
      
      # song id
      song_meta['id'].append(song_id)

      # album name
      album = meta['album']['name']
      song_meta['album']+=[album]

      # song name
      song = meta['name']
      song_meta['name']+=[song]
      
      # artists name
      s = ', '
      artist=s.join([singer_name['name'] for singer_name in meta['artists']])
      song_meta['artist']+=[artist]
      
      # explicit: lyrics could be considered offensive or unsuitable for children
      explicit = meta['explicit']
      song_meta['explicit'].append(explicit)
      
      # song popularity
      popularity = meta['popularity']
      song_meta['popularity'].append(popularity)

  song_meta_df = pd.DataFrame.from_dict(song_meta)

  # check the song feature
  features = sp_access.audio_features(song_meta['id'])
  # change dictionary to dataframe
  features_df = pd.DataFrame.from_dict(features)

  # convert milliseconds to mins
  # duration_ms: The duration of the track in milliseconds.
  # 1 minute = 60 seconds = 60 × 1000 milliseconds = 60,000 ms
  features_df['duration_ms'] = features_df['duration_ms']/60000

  # combine two dataframe
  final_df = song_meta_df.merge(features_df)
  
  final_df.drop(['id', 'analysis_url', 'valence', 'time_signature', 
                 'track_href', 'uri', 'type', 'mode', 
                 'duration_ms', 'key'], 
                axis = 1, 
                inplace = True)

  subset_data = final_df.iloc[:, 4:]

  subset_data_scaled = MinMaxScaler().fit_transform(subset_data)

  subset_data_scaled = pd.DataFrame(subset_data_scaled, 
                                    columns = subset_data.columns)

  return subset_data_scaled

In [ ]:
def playlistColor(results):

  if sys.version_info < (3, 0):
    from urllib2 import urlopen
  else:
      from urllib.request import urlopen

  img_url = ((results).get('images')[0]).get('url')

  color_thief = ColorThief(io.BytesIO(urlopen(img_url).read()))

  playlist_color = '#%02x%02x%02x' % color_thief.get_color(quality=1)    

  return playlist_color

In [ ]:
client_id = '4d500ecf52a3447685e7389ca8a1dae9'
client_secret = 'c5b32293c6fa4339893431280cce8ac1'

sp_access = sp.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id, 
                                                                           client_secret))

In [ ]:
playlist_id_url = input('Enter Playlist Link:\n')

playlist_id = playlist_id_url.split('/')[4]

results = sp_access.playlist(playlist_id)

Enter Playlist Link:
https://open.spotify.com/playlist/37i9dQZF1DWXRqgorJj26U?si=6342f203712d4c91


In [ ]:
playlist_data = get_dataFrame(results, sp_access)

In [ ]:
playlist_data.head(3)

,popularity,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,tempo
0,0.880952,1.000000,0.312966,0.789792,0.360021,0.143497,0.566265,0.222298,0.331426
1,0.988095,0.456061,0.886736,0.913630,0.283645,0.078105,0.002719,0.210437,0.380479
2,0.940476,0.712121,0.771982,0.790308,0.238873,0.102980,0.000000,0.596747,0.360416


In [ ]:
fig = go.Figure(
    data=go.Scatterpolar(
    r=np.int64(np.round(playlist_data.mean() * 100)),
    theta=playlist_data.columns.str.title(),
    fill='toself',
    line=dict(color = playlistColor(results)), 
    )
)

fig.update_layout(
    title = results.get('name'),
    font_size = 15, 
    showlegend=False,
    polar=dict(
        bgcolor = "rgb(236, 240, 241)",
        radialaxis=dict(visible=True),
        ),
)

fig.show()